In [25]:
import fenics as fe
import matplotlib.pyplot as plt
import numpy as np 

In [26]:
from ipywidgets import IntProgress
from IPython.display import display

def log_progress(sequence, every=10):
    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)

    for index, record in enumerate(sequence):
        
        if index % every == 0:
            progress.value = index
        yield record
    progress.value = len(sequence)

In [27]:
L=1
W=0.2
mu=1
rho=1
delta = W/L
gamma=0.4*delta**2
beta=1.25

In [28]:
mesh = fe.BoxMesh(fe.Point(0, 0, 0), fe.Point(L, W, W),10,5,3)
space=fe.VectorFunctionSpace(mesh,'P',1)

In [29]:
Grad=fe.nabla_grad
Div=fe.div
dx=fe.dx
ds=fe.ds
Dot=fe.dot


In [30]:
tol = 1E-14


def claim_bound(x, on_bound):
    return on_bound and x[0] < tol


bc = fe.DirichletBC(space, fe.Constant((0, 0, 0)), claim_bound)

In [44]:
def epsilon_scalar(f):
    # return 0.5 * (Grad(f) + Grad(f).T)
    return fe.sym(fe.nabla_grad(f))


def sigma_vector(f):
    return beta * Dot(Grad(f),Grad(f)) * fe.Identity(3) + 2 * mu * epsilon_scalar(f)


In [45]:
f = fe.TrialFunction(space)
test_f = fe.TestFunction(space)

const_f = fe.Constant((0, 0, -rho * gamma))
const_T = fe.Constant((0, 0, 0))

rhs = fe.inner(sigma_vector(f), epsilon_scalar(f)) * dx
lhs = Dot(const_f, test_f) * dx + Dot(const_T, f) * ds


In [46]:
sol_f=fe.Function(space)
fe.solve(rhs==lhs,sol_f,bc)

        Calling FFC just-in-time (JIT) compiler, this may take some time.


ArityMismatch: Integrand arguments (Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3), 29), VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3)), 0, None),) differ from form arguments (Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3), 29), VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3)), 0, None), Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3), 29), VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3)), 1, None)).